In [1]:
import sys
sys.path.append("../ohw_lter_vis") # go to parent dir
from load_Seward_CTD import make_CTD_dataframe
from load_Seward_zooplankton import make_zooplankton_dataframe

CTD_df = make_CTD_dataframe()
CTD_df.head()

,id,pressure,temperature,temperature2,conductivity,conductivity2,v0,v1,v2,v3,...,instrument,ship,cruise,junk1,PI,purpose,agency,region,junk2,time
0,1,0.0,4.5834,4.5544,2.974495,2.969143,0.2809,0.0005,4.2153,0.0,...,SBE9,R/V Tiglax,TXS12,Hopcroft,Hopcroft,Seward Line Monitoring,NPRB,Gulf of Alaska,n,2012-05-04 05:21:55
1,1,1.0,4.5834,4.5544,2.974495,2.969143,0.2809,0.0005,4.2153,0.0,...,SBE9,R/V Tiglax,TXS12,Hopcroft,Hopcroft,Seward Line Monitoring,NPRB,Gulf of Alaska,n,2012-05-04 05:21:55
2,1,2.0,4.5834,4.5544,2.974495,2.969143,0.2809,0.0005,4.2153,0.0,...,SBE9,R/V Tiglax,TXS12,Hopcroft,Hopcroft,Seward Line Monitoring,NPRB,Gulf of Alaska,n,2012-05-04 05:21:55
3,1,3.0,4.5292,4.5409,2.970221,2.969614,0.2992,0.0005,4.2261,0.0,...,SBE9,R/V Tiglax,TXS12,Hopcroft,Hopcroft,Seward Line Monitoring,NPRB,Gulf of Alaska,n,2012-05-04 05:21:55
4,1,4.0,4.5027,4.5556,2.968559,2.970330,0.2819,0.0006,4.2225,0.0,...,SBE9,R/V Tiglax,TXS12,Hopcroft,Hopcroft,Seward Line Monitoring,NPRB,Gulf of Alaska,n,2012-05-04 05:21:55


In [2]:
import pandas as pd
#infilename='https://workspace.aoos.org/published/file/6c544f8c-6662-4298-bdcf-52029d113c61/Seward_ZooData_Calvet_%202012-2016_final.csv'
infilename='https://workspace.aoos.org/published/file/8453d3df-68c2-42b5-8c17-c17c240b781e/Seward_ZooData_Calvet_2012_2016_final.csv'
zoo_df = pd.read_csv(infilename, header=0, index_col=0)
zoo_df.head()

/opt/conda/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2728: DtypeWarning: Columns (18,19) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,Cruise,Station,Gear Type,Gear Size(m),Tow Type,Net Number,Mesh Size (mm),Latitude (degrees N),Longitude (degrees W),Sample Volume (m3),...,Subphylum,Class,Subclass,Infraclass,Order,Unnamed: 27,Infraorder,Family,Genus,Species Name
1,,,,,,,,,,,,,,,,,,,,,
2,TXS12,GAK1,Multi Net,0.25 m square,Oblique,1,0.5,59.8235,-149.4522,112.4,...,Vertebrata,Actinopterygii,Neopterygii,Teleostei,Osmeriformes,Argentinoidei,NaN,Bathylagidae,NaN,NaN
3,TXS12,GAK1,Multi Net,0.25 m square,Oblique,1,0.5,59.8235,-149.4522,112.4,...,Crustacea,Maxillopoda,Copepoda,Neocopepoda,Calanoida,NaN,NaN,Calanidae,Calanus,marshallae
4,TXS12,GAK1,Multi Net,0.25 m square,Oblique,1,0.5,59.8235,-149.4522,112.4,...,Crustacea,Maxillopoda,Copepoda,Neocopepoda,Calanoida,NaN,NaN,Calanidae,Calanus,marshallae
5,TXS12,GAK1,Multi Net,0.25 m square,Oblique,1,0.5,59.8235,-149.4522,112.4,...,Crustacea,Maxillopoda,Copepoda,Neocopepoda,Calanoida,NaN,NaN,Calanidae,Calanus,marshallae
6,TXS12,GAK1,Multi Net,0.25 m square,Oblique,1,0.5,59.8235,-149.4522,112.4,...,Crustacea,Maxillopoda,Thecostraca,Cirripedia,NaN,NaN,NaN,NaN,NaN,NaN


In [3]:
zoo_df2 = make_zooplankton_dataframe()
zoo_df2.head()

NameError: name 'unicode' is not defined

In [ ]:
import xarray as xr
ds = xr.open_dataset('~/WRF_D02.nc',decode_times=False)
ds.x

In [7]:
infilename='https://workspace.aoos.org/published/file/6c544f8c-6662-4298-bdcf-52029d113c61/Seward_ZooData_Calvet_2012-2016_final.csv'
zooplankton_data = pd.read_csv(infilename, header=0, index_col=0, encoding='latin_1')


In [8]:
zooplankton_data.head()

,Cruise,Date-Time,Year,Month,Day,Time (hh:mm:ss AM/PM),Station,Tow Depth (m),Sonic Depth (m),Latitude (degrees N),...,Unnamed: 73,Unnamed: 74,Unnamed: 75,Unnamed: 76,Unnamed: 77,Unnamed: 78,Unnamed: 79,Unnamed: 80,Unnamed: 81,Unnamed: 82
1,TXS12,5/9/2012 13:45,2012,5,9,13:45,GAK1,100,270,59.8448,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,TXS12,5/9/2012 13:45,2012,5,9,13:45,GAK1,100,270,59.8448,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,TXS12,5/9/2012 13:45,2012,5,9,13:45,GAK1,100,270,59.8448,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,TXS12,5/9/2012 13:45,2012,5,9,13:45,GAK1,100,270,59.8448,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,TXS12,5/9/2012 13:45,2012,5,9,13:45,GAK1,100,270,59.8448,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
